<a href="https://colab.research.google.com/github/gazon1/Recco-challenge/blob/master/lightfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rekko challenge 2019

```
                           /$$$$$$$  /$$$$$$$$ /$$   /$$ /$$   /$$  /$$$$$$ 
                          | $$__  $$| $$_____/| $$  /$$/| $$  /$$/ /$$__  $$
                          | $$  \ $$| $$      | $$ /$$/ | $$ /$$/ | $$  \ $$
                          | $$$$$$$/| $$$$$   | $$$$$/  | $$$$$/  | $$  | $$
                          | $$__  $$| $$__/   | $$  $$  | $$  $$  | $$  | $$
                          | $$  \ $$| $$      | $$\  $$ | $$\  $$ | $$  | $$
                          | $$  | $$| $$$$$$$$| $$ \  $$| $$ \  $$|  $$$$$$/
                          |__/  |__/|________/|__/  \__/|__/  \__/ \______/ 
                                                                            
```

Добро пожаловать на соревнование по машинному обучению от онлайн-кинотеатра [Okko](http://okko.tv) Rekko Challenge 2019.

В этом ноутбуке мы покажем вам пример простого но полного решения, от загрузки данных до формирования ответа. Для работы нам понадобятся библиотеки `pandas`, `numpy`, `scipy`, `implicit`, `pprint`, `tqdm`. Установить их в вашем рабочем окружении можно следующей командой.
```
pip install pandas numpy scipy implicit pprint tqdm
```

In [0]:
import os
import json
import pandas as pd
import numpy as np


import tqdm
import scipy.sparse as sp
from sklearn.metrics import average_precision_score as score
import time
from matplotlib import pyplot as plt

from sklearn import preprocessing
from itertools import chain

from keras.models import model_from_json
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, SpatialDropout1D, Dropout, BatchNormalization, Dense,LSTM, CuDNNGRU
from keras.models import Sequential
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras.models import Model
from keras import optimizers

from tensorflow.python.client import device_lib


from keras import backend as K
K.tensorflow_backend._get_available_gpus()

# Let's set random seed
import tensorflow as tf
tf.set_random_seed(42)
np.random.seed(42)

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer, StandardScaler
import itertools
from pprint import pprint

DATA_PATH = ''
%matplotlib inline

# For Google Colab only:
import sys
sys.path.append('/content/gdrive/My Drive/okko competition')
#from reco_utils.recommender.sar.sar_singlenode import SARSingleNode
from google.colab import drive
drive.mount('/content/gdrive')

DATA_PATH = 'gdrive/My Drive/Colab Notebooks/'
CODE_PATH = 'gdrive/My Drive/okko competition/code'

with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])
    
with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}
catalogue = pd.DataFrame.from_dict(catalogue,'index').reset_index()
# av_hot - нужно для валидации. Эта колонка равна 1, если фильм или сериал может быть потребленным
catalogue['av_hot'] = catalogue['availability'].apply(lambda x: 1 if x == [] or 'subscription' in x else 0)
catalogue = catalogue.rename(index=str, columns={'index':'element_uid'})
catalogue.element_uid = catalogue.element_uid.astype('category')
catalogue['categ_id'] = catalogue.element_uid.cat.codes + 1

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Загрузка данных

Замените `DATA_PATH` на путь к данным, которые вы скачали со страницы соревнования.

In [0]:
transactions = pd.read_csv(
        os.path.join(DATA_PATH, 'transactions.csv'),
        dtype={
            'element_uid': np.uint16,
            'user_uid': np.uint32,
            'consumption_mode': 'category',
            'ts': np.float64,
            'watched_time': np.uint64,
            'device_type': np.uint8,
            'device_manufacturer': np.uint8
        }
    )

ratings = pd.read_csv(
        os.path.join(DATA_PATH, 'ratings.csv'),
        dtype={
            'element_uid': np.uint16,
            'user_uid': np.uint32,
            'ts': np.float64,
            'rating': np.uint8
        }
    )

bookmarks = pd.read_csv(
        os.path.join(DATA_PATH, 'bookmarks.csv'),
        dtype={
            'element_uid': np.uint16,
            'user_uid': np.uint32,
            'ts': np.float64
        }
    )

bookmarks['rating'] = 1
_bookmarks = bookmarks[['rating', 'user_uid', 'element_uid','ts']]
_ratings = ratings[ratings['rating']>5]
_ratings['rating'] = 1

_transaction = transactions[['element_uid', 'user_uid','ts','watched_time']]
_transaction = _transaction[(_transaction['watched_time']>700)]
_transaction['rating'] = 1

all_ratings = pd.concat([ _transaction[['user_uid', 'element_uid', 'rating', 'ts']],_bookmarks[['user_uid', 'element_uid', 'rating', 'ts']]])
all_ratings = pd.concat([all_ratings, _ratings[['user_uid', 'element_uid', 'rating','ts']]])
all_ratings = all_ratings.drop_duplicates(['user_uid','element_uid'],keep='first')
all_ratings = all_ratings.sort_values(by=['ts'],ascending=False)
all_ratings= all_ratings.rename(index=str,columns={'element_uid':'itemID','user_uid':'userID','ts':'timestamp'})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
all_ratings.itemID = all_ratings.itemID.astype('category')
all_ratings['categ_id'] = all_ratings.itemID.cat.codes + 1
cat_to_element_uid = dict(zip(
        range(1, len(all_ratings.itemID.cat.categories) + 1),
        all_ratings.itemID.cat.categories))

all_ratings.itemID = all_ratings.itemID.astype('category')
all_ratings['categ_id'] = all_ratings.itemID.cat.codes + 1
cat_to_element_uid = dict(zip(
        range(1, len(all_ratings.itemID.cat.categories) + 1),
        all_ratings.itemID.cat.categories
    ))

In [0]:
for attr in [7,10,14,15,16,17,18,19,20,21,25,31,32,42,43,52,54,83,270]:
  

  def func(row,attr=attr):
    if attr in row['attributes']:
      return 1
    else:
      return 0

  catalogue['is'+str(attr)] = catalogue.apply(func,axis=1)
catalogue['ismovie'] = 0
catalogue.loc[catalogue['type']=='movie','ismovie'] = 1
catalogue['ismulti'] = 0
catalogue.loc[catalogue['type']=='multipart_movie','ismulti'] = 1
catalogue['isseries'] = 0
catalogue.loc[catalogue['type']=='series','isseries'] = 1

def av_func_p(row):
  if 'purchase' in row['availability']:
    return 1
  else:
    return 0
def av_func_s(row):
  if 'subscription' in row['availability']:
    return 1
  else:
    return 0
def av_func_r(row):
  if 'rent' in row['availability']:
    return 1
  else:
    return 0
  
def av_func_n(row):
  if len(row['availability']) == 0:
    return 1
  else:
    return 0
catalogue['purch'] = catalogue.apply(av_func_p,axis=1)
catalogue['subs'] = catalogue.apply(av_func_s,axis=1)
catalogue['rent'] = catalogue.apply(av_func_r,axis=1)
catalogue['not_known'] = catalogue.apply(av_func_n,axis=1)


In [0]:
catalogue

,element_uid,type,availability,duration,feature_1,feature_2,feature_3,feature_4,feature_5,attributes,...,is54,is83,is270,ismovie,ismulti,isseries,purch,subs,rent,not_known
0,0,movie,[],80,2.912198e+07,0.575260,0,1.128332,0.654707,"[31115, 6713, 10906, 31116, 31117, 270, 24431,...",...,0,0,1,1,0,0,0,0,0,1
1,1,movie,"[purchase, rent]",120,6.610431e+06,0.773224,3,1.112014,0.654707,"[2786, 385, 2799, 3730, 886, 7, 11700, 42, 20,...",...,0,0,0,1,0,0,1,0,1,0
2,2,movie,[],80,1.315874e+07,0.699502,0,1.110127,0.680410,"[31442, 31443, 31444, 31445, 113, 31446, 42, 3...",...,0,0,0,1,0,0,0,0,0,1
3,3,series,[],20,4.157712e+07,0.702981,0,1.141929,0.654707,"[34361, 34362, 23033, 14887, 270, 20089, 43, 25]",...,0,0,1,0,0,1,0,0,0,1
4,4,movie,"[purchase, rent, subscription]",70,3.999579e+07,0.626596,8,1.130076,0.592716,"[26732, 26733, 26734, 9367, 7792, 336, 26735, ...",...,1,0,0,1,0,0,1,1,1,0
5,5,movie,[],100,4.048552e+07,0.693457,0,1.135231,0.680410,"[23108, 23109, 15083, 336, 23110, 123, 42, 43,...",...,0,0,0,1,0,0,0,0,0,1
6,6,movie,"[purchase, rent, subscription]",60,5.896833e+06,0.762507,14,1.121222,0.000000,"[9528, 20806, 20814, 20797, 20815, 270, 20816,...",...,0,0,1,1,0,0,1,1,1,0
7,7,movie,"[purchase, rent, subscription]",80,2.640842e+07,0.751026,12,1.135231,0.449667,"[20418, 20419, 20420, 20421, 20422, 714, 83, 7...",...,0,1,0,1,0,0,1,1,1,0
8,8,movie,"[purchase, rent]",110,7.939682e+06,0.699502,7,1.123020,0.592716,"[172, 416, 2839, 5648, 1183, 7, 5683, 32, 11, ...",...,0,0,0,1,0,0,1,0,1,0
9,9,movie,"[purchase, rent, subscription]",80,1.269700e+07,0.776458,9,0.975056,0.000000,"[16365, 18388, 18389, 18390, 17546, 3271, 396,...",...,0,0,0,1,0,0,1,1,1,0


In [0]:
item_features = catalogue[catalogue['element_uid'].isin(all_ratings['itemID'].values)]
item_features = item_features.drop(['type','availability','element_uid','attributes'],axis=1).values
scaler = StandardScaler()
item_features = scaler.fit_transform(item_features)

item_features = np.hstack((np.identity(item_features.shape[0]),item_features))

In [0]:
item_features.shape

(9566, 9600)

In [0]:
from collections import defaultdict

filtered_elements = defaultdict(set)

for user_uid, element_uid in tqdm.tqdm(transactions.loc[:, ['user_uid', 'element_uid']].values):
    if user_uid not in test_users:
        continue
    filtered_elements[user_uid].add(element_uid)

100%|██████████| 9643012/9643012 [00:18<00:00, 509140.52it/s]


---

---

Для примера мы воспользуемся методом K ближайших соседей, реализованным в библиотеке `implicit`. В качестве данных используем только информацию о рейтингах.

Необходимо построить разреженную матрицу, где строкам будут соответствовать элементы, столбцам — пользователи, а на пересечении пользователя и элемента будет находиться количественная оценка степени их взаимодействия, если таковое имело место.

Не забудем добавить `1` к рейтингу, чтобы избежать деления на ноль во время вычисления `tf-idf`.

In [0]:
all_ratings.head()

,userID,itemID,rating,timestamp,categ_id
0,5177,3336,1,4.430518e+07,3124
1,593316,481,1,4.430518e+07,444
2,262355,4128,1,4.430518e+07,3866
3,74296,6272,1,4.430518e+07,5880
4,340623,5543,1,4.430518e+07,5186


In [0]:
all_ratings['userID'] = all_ratings['userID'].astype('category')
all_ratings['itemID'] = all_ratings['itemID'].astype('category')

ratings_matrix = sp.coo_matrix(
    (all_ratings['rating'].astype(np.float32) + 1,
        (
            all_ratings['itemID'].cat.codes.copy(),
            all_ratings['userID'].cat.codes.copy()
        )
    )
)

ratings_matrix = ratings_matrix.tocsr()
ratings_matrix_T = ratings_matrix.T.tocsr()

In [0]:
sparsity = ratings_matrix.nnz / (ratings_matrix.shape[0] * ratings_matrix.shape[1])
print('Sparsity: %.6f' % sparsity)
ratings_matrix.shape

Sparsity: 0.001751


(9566, 496046)

Обучить модель крайне просто.

Отображения из оригинальной категории во внутреннюю пригодится нам в дальнейшем.

In [0]:
user_uid_to_cat = dict(zip(
    all_ratings['userID'].cat.categories,
    range(len(all_ratings['userID'].cat.categories))
))

element_uid_to_cat = dict(zip(
    all_ratings['itemID'].cat.categories,
    range(len(all_ratings['itemID'].cat.categories))
))


filtered_elements_cat = {k: [element_uid_to_cat.get(x, None) for x in v] for k, v in filtered_elements.items()}

In [0]:
# основная функция для валидации модели
def validation(dict_):
  d1 = {}
  with open(DATA_PATH+'my_best.json', 'r') as f:
    d1 = json.load(f)
  av = []


  for key in d1.keys():
      if key in dict_.keys():
          ar1 = np.array(d1[key])
         # print(ar1)
          ar2 = np.array(dict_[key])
          inter = np.intersect1d(ar1,ar2)
          av.append(len(inter))
          to_add = []
          for el in ar1:
              if el  in inter:
                  to_add.append(el)
          for el in ar1:
              if el  not in to_add:
                  to_add.append(el)


          to_add = np.array(to_add)
  return np.mean(av),np.var(av)

def shuffle(dict_):
  d = {}
  d1 = {}
  with open(DATA_PATH+'my_best.json', 'r') as f:
    d1 = json.load(f)
  
  for key in dict_.keys():
    if key in d1.keys():
      ar1 = np.array(dict_[key])
       # print(ar1)
      ar2 = np.array(d1[key])
      inter = np.intersect1d(ar1,ar2)

      to_add = []
      for el in ar1:
        if el  in inter:
          to_add.append(el)
      for el in ar1:
        if el  not in to_add:
          to_add.append(el)
        
        
      to_add = np.array(to_add)
        
      d[key] = list(map(int,list(to_add)))

    else:
      d[key] = dict_[key]
      
  return d
       


In [0]:
!pip3 install lightfm

    100% |████████████████████████████████| 307kB 7.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [0]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from scipy import sparse
from lightfm.datasets import fetch_movielens

In [0]:
#%%time
model = LightFM(loss='warp',no_components=15)
#model.fit(ratings_matrix_T, epochs=10, num_threads=4,verbose=True)

model.fit(ratings_matrix_T, item_features=sparse.csr_matrix(item_features), epochs=50, num_threads=4,verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


In [0]:
d1 = {}
t = sparse.csr_matrix(item_features)
for user in tqdm.tqdm(list(test_users)):
   # print(user)
  try:
    cats = list(np.argsort(-model.predict(user_uid_to_cat[int(user)],[i for i in element_uid_to_cat.values()],\
                                           item_features=t,
                                          num_threads=30  )))[:80]

    d1[str(user)] = [int(all_ratings['itemID'].cat.categories[i]) for i in cats if i not in filtered_elements_cat[user]]
      #print(d[str(user)])
    d1[str(user)] = d1[str(user)][:20]
  except:
    pass
print(validation(d1))

to_send = shuffle(d1)
    


In [0]:
with open(os.path.join(DATA_PATH, 'answer_50.json'), 'w') as f:
    json.dump(to_send, f)

In [0]:
np.argsort(-model.predict(user_uid_to_cat[13],[i for i in element_uid_to_cat.values()],\
                                           item_features=t,num_threads=30  ))

array([ 197,  117,  513, ..., 9038, 8921, 9261])

In [0]:
len(to_send)

49895